In [1]:
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
import os
import glob
import calendar as cal
from array import array 


In [2]:
# Combine individual GEFS files for each start into one master netcdf file
# Each file is a start day (IC at 0z), going out 64 leads, every 6hrs (64 leads/4 = 16 days)
# 11 Ensemble members

# Isolate just the U.S. domain to reduce file size

# Reduce lead time to 16 points/days by SUMMING 4 leads together (0Z to 0Z)

# Also isolate one variable (accumulated precip)

# Loop through specified year /month /days
# 1985 to 2015 are complete years with 11 members

obj = cal.Calendar() 

ds_Y = []
#for iy in np.arange(1985, 2016):
for iy in np.arange(1985, 1986):
    ds_M = []
    # Loop through only JJA months 
    #for imth in np.arange(1,13):
    for imth in np.arange(6,9):
        # Extract last day of each month and then format month
        monthdays = [d for d in obj.itermonthdays(iy, imth) if d != 0]
        lstdy = monthdays[-1]
        imth = "{0:0=2d}".format(imth)
        #print(lstdy)
        pr_S_list = []
        for idy in np.arange(0, lstdy):
        #for idy in np.arange(0, 2):
            test = monthdays[idy]
            nw = int(test)
            idy = "{0:0=2d}".format(nw)
            print(idy)
            filname='gefs_'+np.str(iy)+np.str(imth)+np.str(idy)+'_00z.nc'
            ifilename='/cpc/model_reforecasts/gefs-v10/'+np.str(iy)+'/'+np.str(imth)+'/'+np.str(idy)+'/00/'+np.str(filname)
            #print(np.str(filname))
            print(ifilename)
            ds = xr.open_dataset(ifilename)
            # select variable
            prcp = ds.APCP_P11_L1_GLL0_acc
            # Overwrite with new date range
            tim=np.str(iy)+'-'+np.str(imth)+'-'+np.str(idy)
            newtim = pd.date_range(tim, periods=64, freq='6H')
            prcp['forecast_time1'] = newtim
            # Average into daily
            daily = prcp.resample(forecast_time1='D').sum(dim='forecast_time1')
            # Overwrite the lead time (days- 1 through 16) or else files won't concatnate properly
            tim2=np.arange(1,17)
            daily['forecast_time1'] = tim2
            #print(daily.forecast_time1)
            us = daily.sel(lat_0=slice(75,20),lon_0=slice(-170+360,-50+360))
            us.coords['S'] = tim
            us = us.expand_dims('S')
            del daily
            pr_S_list.append(us)
        pr_S = xr.concat(pr_S_list,dim='S') # Concats days
        ds_M.append(pr_S)
        del pr_S_list
    all_M = xr.concat(ds_M, dim='S') # Concats months
    ds_Y.append(all_M)
    del ds_M
all_Y = xr.concat(ds_Y, dim='S') # Concats years
del ds_Y



01
/cpc/model_reforecasts/gefs-v10/1985/06/01/00/gefs_19850601_00z.nc
02
/cpc/model_reforecasts/gefs-v10/1985/06/02/00/gefs_19850602_00z.nc
03
/cpc/model_reforecasts/gefs-v10/1985/06/03/00/gefs_19850603_00z.nc
04
/cpc/model_reforecasts/gefs-v10/1985/06/04/00/gefs_19850604_00z.nc
05
/cpc/model_reforecasts/gefs-v10/1985/06/05/00/gefs_19850605_00z.nc
06
/cpc/model_reforecasts/gefs-v10/1985/06/06/00/gefs_19850606_00z.nc
07
/cpc/model_reforecasts/gefs-v10/1985/06/07/00/gefs_19850607_00z.nc
08
/cpc/model_reforecasts/gefs-v10/1985/06/08/00/gefs_19850608_00z.nc
09
/cpc/model_reforecasts/gefs-v10/1985/06/09/00/gefs_19850609_00z.nc
10
/cpc/model_reforecasts/gefs-v10/1985/06/10/00/gefs_19850610_00z.nc
11
/cpc/model_reforecasts/gefs-v10/1985/06/11/00/gefs_19850611_00z.nc
12
/cpc/model_reforecasts/gefs-v10/1985/06/12/00/gefs_19850612_00z.nc
13
/cpc/model_reforecasts/gefs-v10/1985/06/13/00/gefs_19850613_00z.nc
14
/cpc/model_reforecasts/gefs-v10/1985/06/14/00/gefs_19850614_00z.nc
15
/cpc/model_refore

In [3]:
#print(pr_S.sizes)
#print(pr_S.coords)
#print(pr_S.forecast_time1)

print(all_Y.sizes)
print(all_Y.coords)

Frozen({'S': 92, 'ensemble0': 11, 'forecast_time1': 16, 'lat_0': 56, 'lon_0': 121})
Coordinates:
  * lon_0           (lon_0) float32 190.0 191.0 192.0 ... 308.0 309.0 310.0
  * ensemble0       (ensemble0) int32 0 1 2 3 4 5 6 7 8 9 10
  * forecast_time1  (forecast_time1) int64 1 2 3 4 5 6 7 ... 11 12 13 14 15 16
  * lat_0           (lat_0) float32 75.0 74.0 73.0 72.0 ... 23.0 22.0 21.0 20.0
  * S               (S) object '1985-06-01' '1985-06-02' ... '1985-08-31'


In [4]:
all_Y.to_netcdf('All_GEFS_Prcp.nc')

In [ ]:
# Open netcdf file
#ds = xr.open_dataset('All_GEFS_Prcp.nc')
#ds